In [29]:
import shutil
import tqdm
import numpy as np
import cv2
import os
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
import numpy as np
import os, sys
import pickle, functools, operator
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import joblib
from tensorflow.keras.utils import to_categorical
from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
import json
import random
from keras.utils.vis_utils import plot_model
import tensorflow as tf
import datetime

from pandas import read_csv
import numpy as np
from keras import Model
from keras.layers import Layer
import keras.backend as K
from keras.layers import Input, Dense, SimpleRNN , Concatenate
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.metrics import mean_squared_error

import random
random.seed(42)

In [30]:
class config():
    train_path = "./training_data"
    test_path = "./testing_data"
    batch_size = 160
    learning_rate = 0.0007
    epochs = 150
    latent_dim = 512
    num_encoder_tokens = 4096
    num_decoder_tokens = 1500
    time_steps_encoder = 80
    max_probability = -1
    save_model_path = 'model_final'
    validation_split = 0.15
    max_length = 10
    search_type = 'greedy'

In [31]:
config = config()

## 01. Train & Test Split Data

In [32]:
with open('./training_data/AllVideoDescriptions.txt') as f:
    lines = f.readlines()

In [33]:
all_videos = list(set([l.split(' ')[0] for l in lines]))

In [34]:
with open('./train_split.txt','r') as f:
    train_l = f.readlines()
train_l = [l.replace('\n','') for l in train_l]

with open('./test_split.txt','r') as f:
    test_l = f.readlines()
test_l = [l.replace('\n','') for l in test_l]

with open('./val_split.txt','r') as f:
    val_l = f.readlines()
val_l = [l.replace('\n','') for l in val_l]

In [35]:
len(train_l), len(test_l) , len(val_l)

(1200, 670, 100)

In [36]:
scripts = {}
for l in lines:
    id = l.split(' ')[0]
    script = ' '.join(l.split(' ')[1:])
    if id in scripts:
      scripts[id].append(script)
    else:
      scripts[id] = [script]

In [93]:
training_list = []
validation_list = []
test_list = [] 
vocab_list = []

for y in train_l:
  for caption in scripts[y][0:1]:
    caption = "<bos> " + caption + " <eos>"

    training_list.append([caption, y])

for y in val_l:
  for caption in scripts[y][0:1]:
    caption = "<bos> " + caption + " <eos>"

    validation_list.append([caption, y])
    
for y in test_l:
  for caption in scripts[y][0:1]:
    caption = "<bos> " + caption + " <eos>"

    test_list.append([caption, y])

In [95]:
train_list = training_list + validation_list + test_list

In [96]:
print(len(train_list))
random.shuffle(training_list)
random.shuffle(validation_list)
random.shuffle(test_list)

for train in training_list:
    vocab_list.append(train[0])
# Tokenizing the words
tokenizer = Tokenizer(num_words=1500)
tokenizer.fit_on_texts(vocab_list)
print(len(tokenizer.word_index))
x_data = {}
TRAIN_FEATURE_DIR = os.path.join('training_data', 'features_dir')
# Loading all the numpy arrays at once and saving them in a dictionary
for filename in os.listdir(TRAIN_FEATURE_DIR):
    f = np.load(os.path.join(TRAIN_FEATURE_DIR, filename))
    x_data[filename[:-4]] = f
print(len(training_list))
print(len(validation_list))
len(x_data)

1970
1702
1200
100


1970

## 02. Model Training

In [98]:
# Creating a custom data generator because we cannot load so many files at once
def load_datatest(train_path, epochs=100, x_data=x_data, tokenizer=tokenizer, num_decoder_tokens=1500,training_list=training_list, batch_size=32, maxlen=30):
    encoder_input_data = []
    decoder_input_data = []
    decoder_target_data = []
    videoId = []
    videoSeq = []
    # separating the videoId and the video captions
    for idx, cap in enumerate(training_list):
        caption = cap[0]
        videoId.append(cap[1])
        videoSeq.append(caption)
    # converting the captions to tokens and padding them to equal sizes
    train_sequences = tokenizer.texts_to_sequences(videoSeq)
    train_sequences = np.array(train_sequences)
    train_sequences = pad_sequences(train_sequences, padding='post',truncating='post', maxlen=maxlen)
    max_seq_length = train_sequences.shape[1]
    filesize = len(train_sequences)
    vCount = 0
    n = 0
    for i in range(epochs):
      for idx in  range(0,filesize):
        n += 1
        try:
          encoder_input_data.append(x_data[videoId[idx]])
        except:
          continue
        y = to_categorical(train_sequences[idx], num_decoder_tokens)
        decoder_input_data.append(y[:-1])
        decoder_target_data.append(y[1:])
        if n == batch_size:
          encoder_input = np.array(encoder_input_data)
          decoder_input = np.array(decoder_input_data)
          decoder_target = np.array(decoder_target_data)
          encoder_input_data = []
          decoder_input_data = []
          decoder_target_data = []
          n = 0
          yield ([encoder_input, decoder_input], decoder_target)


In [103]:
# writing the train and validation generator
train = load_datatest(train_path='training_data',batch_size=320, training_list=training_list, x_data=x_data, epochs=150)
valid = load_datatest(train_path='training_data',batch_size=320, training_list=validation_list, x_data=x_data, epochs=150)
test = load_datatest(train_path='test_data',batch_size=320, training_list=test_list, x_data=x_data, epochs=150)

In [104]:
"""
time_steps_encoder is the number of frames per video we will be using for training
num_encoder_tokens is the number of features from each frame
latent_dim is the number of hidden features for lstm
time_steps_decoder is the maximum length of each sentence
num_decoder_tokens is the final number of tokens in the softmax layer
batch size
"""
time_steps_encoder=80
num_encoder_tokens=4096
latent_dim=512
time_steps_decoder=30
num_decoder_tokens=1500
batch_size=320

# Setting up the encoder
encoder_inputs = Input(shape=(time_steps_encoder, num_encoder_tokens), name="encoder_inputs")
encoder = LSTM(latent_dim, return_state=True,return_sequences=True, name='endcoder_lstm')
_, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]


# Set up the decoder
decoder_inputs = Input(shape=(time_steps_decoder,num_decoder_tokens), name= "decoder_inputs")
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder_lstm')
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_relu')
decoder_outputs = decoder_dense(decoder_outputs)


model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()
plot_model(model, to_file='model_train.png', show_shapes=True, show_layer_names=True)

Model: "model_16"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, 80, 4096)]   0           []                               
                                                                                                  
 decoder_inputs (InputLayer)    [(None, 30, 1500)]   0           []                               
                                                                                                  
 endcoder_lstm (LSTM)           [(None, 80, 512),    9439232     ['encoder_inputs[0][0]']         
                                 (None, 512),                                                     
                                 (None, 512)]                                                     
                                                                                           

In [105]:
# Early Stopping
earlystopping = EarlyStopping(monitor='val_loss', patience = 5, verbose=1, mode='min')

# Tensorboard callback
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
#tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

# # Run training
opt = tf.keras.optimizers.Adam(lr = 0.0003)
x = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.1,patience=2,verbose=0,mode="auto")
model.compile(metrics=['accuracy'], optimizer=opt, loss='categorical_crossentropy')


try:
    model.fit(train, validation_data=valid, validation_steps=(len(validation_list)//batch_size),
        epochs=config.epochs, steps_per_epoch=(len(training_list)//batch_size),
            callbacks=[x, earlystopping])
except KeyboardInterrupt:
    print("\nW: interrupt received, stopping")
finally:
    pass

/tmp/ipykernel_785316/3020350894.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_sequences = np.array(train_sequences)


Epoch 1/150
3/3 [==============================] - 3s 713ms/step - loss: 7.1023 - accuracy: 0.4035 - lr: 3.0000e-04
Epoch 2/150
3/3 [==============================] - 2s 602ms/step - loss: 5.9345 - accuracy: 0.6969 - lr: 3.0000e-04
Epoch 3/150
3/3 [==============================] - 2s 606ms/step - loss: 4.0857 - accuracy: 0.6977 - lr: 3.0000e-04
Epoch 4/150
3/3 [==============================] - 2s 618ms/step - loss: 2.8212 - accuracy: 0.6916 - lr: 3.0000e-04
Epoch 5/150
3/3 [==============================] - 2s 533ms/step - loss: 2.2397 - accuracy: 0.6967 - lr: 3.0000e-04
Epoch 6/150
3/3 [==============================] - 2s 523ms/step - loss: 2.1392 - accuracy: 0.6936 - lr: 3.0000e-04
Epoch 7/150
3/3 [==============================] - 2s 512ms/step - loss: 2.0050 - accuracy: 0.7043 - lr: 3.0000e-04
Epoch 8/150
3/3 [==============================] - 2s 623ms/step - loss: 1.9329 - accuracy: 0.7020 - lr: 3.0000e-04
Epoch 9/150
3/3 [==============================] - 1s 512ms/step - loss:

3/3 [==============================] - 2s 617ms/step - loss: 1.7584 - accuracy: 0.7093 - lr: 3.0000e-04
Epoch 19/150
3/3 [==============================] - 2s 635ms/step - loss: 1.7904 - accuracy: 0.7045 - lr: 3.0000e-04
Epoch 20/150
3/3 [==============================] - 2s 623ms/step - loss: 1.7388 - accuracy: 0.7092 - lr: 3.0000e-04
Epoch 21/150
3/3 [==============================] - 1s 447ms/step - loss: 1.8061 - accuracy: 0.7014 - lr: 3.0000e-04
Epoch 22/150
3/3 [==============================] - 2s 622ms/step - loss: 1.7264 - accuracy: 0.7084 - lr: 3.0000e-04
Epoch 23/150
3/3 [==============================] - 1s 517ms/step - loss: 1.6977 - accuracy: 0.7115 - lr: 3.0000e-04
Epoch 24/150
3/3 [==============================] - 2s 598ms/step - loss: 1.7302 - accuracy: 0.7063 - lr: 3.0000e-04
Epoch 25/150
3/3 [==============================] - 2s 528ms/step - loss: 1.6798 - accuracy: 0.7108 - lr: 3.0000e-04
Epoch 26/150
3/3 [==============================] - 2s 605ms/step - loss: 1.7

Epoch 36/150
3/3 [==============================] - 1s 551ms/step - loss: 1.6437 - accuracy: 0.7150 - lr: 3.0000e-04
Epoch 37/150
3/3 [==============================] - 2s 509ms/step - loss: 1.5689 - accuracy: 0.7213 - lr: 3.0000e-04
Epoch 38/150
3/3 [==============================] - 2s 526ms/step - loss: 1.5423 - accuracy: 0.7288 - lr: 3.0000e-04
Epoch 39/150
3/3 [==============================] - 2s 512ms/step - loss: 1.5751 - accuracy: 0.7214 - lr: 3.0000e-04
Epoch 40/150
3/3 [==============================] - 2s 597ms/step - loss: 1.5264 - accuracy: 0.7289 - lr: 3.0000e-04
Epoch 41/150
3/3 [==============================] - 1s 532ms/step - loss: 1.5827 - accuracy: 0.7249 - lr: 3.0000e-04
Epoch 42/150
3/3 [==============================] - 2s 619ms/step - loss: 1.5100 - accuracy: 0.7314 - lr: 3.0000e-04
Epoch 43/150
3/3 [==============================] - 2s 595ms/step - loss: 1.4849 - accuracy: 0.7379 - lr: 3.0000e-04
Epoch 44/150
3/3 [==============================] - 2s 607ms/ste

3/3 [==============================] - 1s 554ms/step - loss: 1.4625 - accuracy: 0.7423 - lr: 3.0000e-04
Epoch 54/150
3/3 [==============================] - 2s 514ms/step - loss: 1.4523 - accuracy: 0.7420 - lr: 3.0000e-04
Epoch 55/150
3/3 [==============================] - 2s 426ms/step - loss: 1.4043 - accuracy: 0.7454 - lr: 3.0000e-04
Epoch 56/150
3/3 [==============================] - 2s 538ms/step - loss: 1.4779 - accuracy: 0.7394 - lr: 3.0000e-04
Epoch 57/150
3/3 [==============================] - 2s 563ms/step - loss: 1.4048 - accuracy: 0.7442 - lr: 3.0000e-04
Epoch 58/150
3/3 [==============================] - 2s 428ms/step - loss: 1.3712 - accuracy: 0.7523 - lr: 3.0000e-04
Epoch 59/150
3/3 [==============================] - 2s 431ms/step - loss: 1.4001 - accuracy: 0.7400 - lr: 3.0000e-04
Epoch 60/150
3/3 [==============================] - 2s 608ms/step - loss: 1.3597 - accuracy: 0.7550 - lr: 3.0000e-04
Epoch 61/150
3/3 [==============================] - 1s 515ms/step - loss: 1.4

Epoch 71/150
3/3 [==============================] - 2s 545ms/step - loss: 1.3352 - accuracy: 0.7566 - lr: 3.0000e-04
Epoch 72/150
3/3 [==============================] - 2s 520ms/step - loss: 1.2762 - accuracy: 0.7543 - lr: 3.0000e-04
Epoch 73/150
3/3 [==============================] - 2s 622ms/step - loss: 1.2724 - accuracy: 0.7594 - lr: 3.0000e-04
Epoch 74/150
3/3 [==============================] - 1s 529ms/step - loss: 1.2999 - accuracy: 0.7591 - lr: 3.0000e-04
Epoch 75/150
3/3 [==============================] - 2s 535ms/step - loss: 1.2648 - accuracy: 0.7541 - lr: 3.0000e-04
Epoch 76/150
3/3 [==============================] - 2s 537ms/step - loss: 1.3325 - accuracy: 0.7525 - lr: 3.0000e-04
Epoch 77/150
3/3 [==============================] - 2s 431ms/step - loss: 1.2568 - accuracy: 0.7635 - lr: 3.0000e-04
Epoch 78/150
3/3 [==============================] - 2s 412ms/step - loss: 1.2277 - accuracy: 0.7575 - lr: 3.0000e-04
Epoch 79/150
3/3 [==============================] - 2s 526ms/ste

3/3 [==============================] - 2s 522ms/step - loss: 1.1577 - accuracy: 0.7737 - lr: 3.0000e-04
Epoch 89/150
3/3 [==============================] - 2s 612ms/step - loss: 1.1788 - accuracy: 0.7632 - lr: 3.0000e-04
Epoch 90/150
3/3 [==============================] - 1s 520ms/step - loss: 1.1589 - accuracy: 0.7765 - lr: 3.0000e-04
Epoch 91/150
3/3 [==============================] - 2s 517ms/step - loss: 1.2331 - accuracy: 0.7607 - lr: 3.0000e-04
Epoch 92/150
3/3 [==============================] - 2s 527ms/step - loss: 1.1720 - accuracy: 0.7627 - lr: 3.0000e-04
Epoch 93/150
3/3 [==============================] - 2s 403ms/step - loss: 1.1686 - accuracy: 0.7724 - lr: 3.0000e-04
Epoch 94/150
3/3 [==============================] - 2s 616ms/step - loss: 1.1887 - accuracy: 0.7624 - lr: 3.0000e-04
Epoch 95/150
3/3 [==============================] - 2s 603ms/step - loss: 1.1338 - accuracy: 0.7728 - lr: 3.0000e-04
Epoch 96/150
3/3 [==============================] - 2s 602ms/step - loss: 1.1

Epoch 106/150
3/3 [==============================] - 2s 518ms/step - loss: 1.1136 - accuracy: 0.7646 - lr: 3.0000e-04
Epoch 107/150
3/3 [==============================] - 2s 614ms/step - loss: 1.0644 - accuracy: 0.7876 - lr: 3.0000e-04
Epoch 108/150
3/3 [==============================] - 1s 521ms/step - loss: 1.0530 - accuracy: 0.7810 - lr: 3.0000e-04
Epoch 109/150
3/3 [==============================] - 2s 603ms/step - loss: 1.0731 - accuracy: 0.7775 - lr: 3.0000e-04
Epoch 110/150
3/3 [==============================] - 2s 639ms/step - loss: 1.0555 - accuracy: 0.7836 - lr: 3.0000e-04
Epoch 111/150
3/3 [==============================] - 1s 559ms/step - loss: 1.0954 - accuracy: 0.7676 - lr: 3.0000e-04
Epoch 112/150
3/3 [==============================] - 2s 662ms/step - loss: 1.0409 - accuracy: 0.7934 - lr: 3.0000e-04
Epoch 113/150
3/3 [==============================] - 2s 619ms/step - loss: 1.0297 - accuracy: 0.7770 - lr: 3.0000e-04
Epoch 114/150
3/3 [==============================] - 2s 

3/3 [==============================] - 2s 603ms/step - loss: 0.9623 - accuracy: 0.8030 - lr: 3.0000e-04
Epoch 124/150
3/3 [==============================] - 2s 605ms/step - loss: 0.9830 - accuracy: 0.7847 - lr: 3.0000e-04
Epoch 125/150
3/3 [==============================] - 1s 522ms/step - loss: 0.9521 - accuracy: 0.7974 - lr: 3.0000e-04
Epoch 126/150
3/3 [==============================] - 2s 449ms/step - loss: 0.9927 - accuracy: 0.7866 - lr: 3.0000e-04
Epoch 127/150
3/3 [==============================] - 2s 438ms/step - loss: 0.9426 - accuracy: 0.7944 - lr: 3.0000e-04
Epoch 128/150
3/3 [==============================] - 2s 626ms/step - loss: 0.9254 - accuracy: 0.8060 - lr: 3.0000e-04
Epoch 129/150
3/3 [==============================] - 2s 547ms/step - loss: 0.9444 - accuracy: 0.7901 - lr: 3.0000e-04
Epoch 130/150
3/3 [==============================] - 2s 626ms/step - loss: 0.9145 - accuracy: 0.8072 - lr: 3.0000e-04
Epoch 131/150
3/3 [==============================] - 2s 609ms/step - l

Epoch 141/150
3/3 [==============================] - 2s 513ms/step - loss: 0.8787 - accuracy: 0.8085 - lr: 3.0000e-04
Epoch 142/150
3/3 [==============================] - 2s 521ms/step - loss: 0.8367 - accuracy: 0.8150 - lr: 3.0000e-04
Epoch 143/150
3/3 [==============================] - 2s 634ms/step - loss: 0.8237 - accuracy: 0.8149 - lr: 3.0000e-04
Epoch 144/150
3/3 [==============================] - 1s 435ms/step - loss: 0.8440 - accuracy: 0.8138 - lr: 3.0000e-04
Epoch 145/150
3/3 [==============================] - 2s 439ms/step - loss: 0.8266 - accuracy: 0.8143 - lr: 3.0000e-04
Epoch 146/150
3/3 [==============================] - 2s 616ms/step - loss: 0.8636 - accuracy: 0.8005 - lr: 3.0000e-04
Epoch 147/150
3/3 [==============================] - 2s 610ms/step - loss: 0.8475 - accuracy: 0.8021 - lr: 3.0000e-04
Epoch 148/150
3/3 [==============================] - 2s 532ms/step - loss: 0.8495 - accuracy: 0.8027 - lr: 3.0000e-04
Epoch 149/150
3/3 [==============================] - 2s 

## 03. SaveModel

In [64]:
x = datetime.datetime.now()

save_model_path = 'model_final' + str(x)[:11]
print(f"Save model path is : {save_model_path}")

if not os.path.exists(save_model_path):
    os.makedirs(save_model_path)

# Saving encoder as in training
encoder_model = Model(encoder_inputs, encoder_states)

#Saving decoder states and dense layer 
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

encoder_model.summary()
decoder_model.summary()
encoder_model.save(os.path.join(save_model_path, 'encoder_model.h5'))
decoder_model.save_weights(os.path.join(save_model_path, 'decoder_model_weights.h5'))
with open(os.path.join(save_model_path,'tokenizer'+ str(num_decoder_tokens) ),'wb') as file:
    joblib.dump(tokenizer, file)
plot_model(encoder_model, to_file='model_inference_encoder.png', show_shapes=True, show_layer_names=True)
plot_model(decoder_model, to_file='model_inference_decoder.png', show_shapes=True, show_layer_names=True)

Save model path is : model_final2022-06-03 
Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_inputs (InputLayer)  [(None, 80, 4096)]       0         
                                                                 
 endcoder_lstm (LSTM)        [(None, 80, 512),         9439232   
                              (None, 512),                       
                              (None, 512)]                       
                                                                 
Total params: 9,439,232
Trainable params: 9,439,232
Non-trainable params: 0
_________________________________________________________________
Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 decoder_inputs (InputLayer)    [(None, 30, 1500)]   0

In [65]:
with open(os.path.join(save_model_path,'tokenizer'+ str(num_decoder_tokens) ),'wb') as file:
    joblib.dump(tokenizer, file)

## 04. Trained-model Load & Inference

In [76]:
# class to perform inference on all test files and save as test_output.txt
class Video2Text(object):
    ''' Initialize the parameters for the model '''
    def __init__(self):
        self.latent_dim = 512
        self.num_encoder_tokens = 4096
        self.num_decoder_tokens = 1500
        self.time_steps_encoder = 80
        self.time_steps_decoder = None
        self.preload = True
        self.preload_data_path = 'preload_data'
        self.max_probability = -1

        # processed data
        self.encoder_input_data = []
        self.decoder_input_data = []
        self.decoder_target_data = []
        self.tokenizer = None

        # models
        self.encoder_model = None
        self.decoder_model = None
        self.inf_encoder_model = None
        self.inf_decoder_model = None
        self.save_model_path = save_model_path
        self.test_path = 'testing_data'
        
        
    def load_inference_models(self):
        # load tokenizer
        
        with open(os.path.join(self.save_model_path, 'tokenizer' + str(self.num_decoder_tokens)), 'rb') as file:
            self.tokenizer = joblib.load(file)

        # inference encoder model
        self.inf_encoder_model = load_model(os.path.join(self.save_model_path, 'encoder_model.h5'),compile=False)
        self.inf_encoder_model.summary()
        # inference decoder model
        decoder_inputs = Input(shape=(None, self.num_decoder_tokens))
        decoder_lstm = LSTM(self.latent_dim, return_sequences=True, return_state=True)
        decoder_state_input_h = Input(shape=(latent_dim,))
        decoder_state_input_c = Input(shape=(latent_dim,))
        decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
        decoder_dense = Dense(self.num_decoder_tokens, activation='softmax', name='decoder_relu')
        decoder_outputs, state_h, state_c = decoder_lstm(
            decoder_inputs, initial_state=decoder_states_inputs)
        decoder_states = [state_h, state_c]
        decoder_outputs = decoder_dense(decoder_outputs)
        self.inf_decoder_model = Model(
            [decoder_inputs] + decoder_states_inputs,
            [decoder_outputs] + decoder_states)
#         self.inf_decoder_model = Model(
#             [decoder_inputs] + decoder_states_inputs,
#             [decoder_outputs] + decoder_states)
        self.inf_decoder_model.summary()
        self.inf_decoder_model.load_weights(os.path.join(self.save_model_path, 'decoder_model_weights.h5'))
    
    def decode_sequence2bs(self, input_seq):
        states_value = self.inf_encoder_model.predict(input_seq)
        target_seq = np.zeros((1, 1, self.num_decoder_tokens))
        target_seq[0, 0, self.tokenizer.word_index['bos']] = 1
        self.beam_search(target_seq, states_value,[],[],0)
        return decode_seq

    def beam_search(self, target_seq, states_value, prob,  path, lens):
        global decode_seq
        node = 2
        output_tokens, h, c = self.inf_decoder_model.predict(
            [target_seq] + states_value)
        output_tokens = output_tokens.reshape((self.num_decoder_tokens))
        sampled_token_index = output_tokens.argsort()[-node:][::-1]
        states_value = [h, c]
        for i in range(node):
            if sampled_token_index[i] == 0:
                sampled_char = ''
            else:
                sampled_char = list(self.tokenizer.word_index.keys())[list(self.tokenizer.word_index.values()).index(sampled_token_index[i])]
            MAX_LEN = 10
            if(sampled_char != 'eos' and lens <= MAX_LEN):
                p = output_tokens[sampled_token_index[i]]
                if(sampled_char == ''):
                    p = 1
                prob_new = list(prob)
                prob_new.append(p)
                path_new = list(path)
                path_new.append(sampled_char)
                target_seq = np.zeros((1, 1, self.num_decoder_tokens))
                target_seq[0, 0, sampled_token_index[i]] = 1.
                self.beam_search(target_seq, states_value, prob_new, path_new, lens+1)
            else:
                p = output_tokens[sampled_token_index[i]]
                prob_new = list(prob)
                prob_new.append(p)
                p = functools.reduce(operator.mul, prob_new, 1)
                if(p > self.max_probability):
                    decode_seq = path
                    self.max_probability = p

    def decoded_sentence_tuning(self, decoded_sentence):
        decode_str = []
        filter_string = ['bos', 'eos']
        unigram = {}
        last_string = ""
        for idx2, c in enumerate(decoded_sentence):
            if c in unigram:
                unigram[c] += 1
            else:
                unigram[c] = 1
            if(last_string == c and idx2 > 0):
                continue
            if c in filter_string:
                continue
            if len(c) > 0:
                decode_str.append(c)
            if idx2 > 0:
                last_string = c
        return decode_str

    def get_test_data(self, path):
        X_test = []
        X_test_filename = []
#         with open (os.path.join(path, 'testing_id.txt')) as testing_file:
#             lines = testing_file.readlines()
#             for filename in lines:
#                 filename = filename.strip()
#                 f = np.load(os.path.join(path , 'feat', filename + '.npy'))
#                 X_test.append(f)
#                 X_test_filename.append(filename[:-4])
#             X_test = np.array(X_test)
        for filename in test_l:
            try:
                f = np.load(os.path.join(path , 'features_dir', filename + '.npy'))
            except: 
                print('not exsits')
                continue
            X_test.append(f)
            X_test_filename.append(filename)
        return X_test, X_test_filename

    def test(self):
        X_test, X_test_filename = self.get_test_data(os.path.join(config.train_path)) 
        self.X_test , self.X_test_filename = X_test, X_test_filename
        # generate inference test outputs
        with open(os.path.join(self.save_model_path, 'test_output.txt'), 'w') as file:
            for idx, x in enumerate(X_test): 
                file.write(X_test_filename[idx]+',')
                decoded_sentence = self.decode_sequence2bs(x.reshape(-1, 80, 4096))
                decode_str = self.decoded_sentence_tuning(decoded_sentence)
                for d in decode_str:
                    file.write(d + ' ')
                #file.write(decoded_sentence+' ')
                file.write('\n')
                # re-init max prob
                self.max_probability = -1
                
    def index_to_word(self):
        # inverts word tokenizer
        index_to_word = {value: key for key, value in self.tokenizer.word_index.items()}
        return index_to_word
                
    def greedy_search(self, f):
        """
                :param f: the loaded numpy array after creating videos to frames and extracting features
                :return: the final sentence which has been predicted greedily
                """
        inv_map = self.index_to_word()
        states_value = self.inf_encoder_model.predict(f.reshape(-1, 80, 4096))
        target_seq = np.zeros((1, 1, 1500))
        sentence = ''
        target_seq[0, 0, self.tokenizer.word_index['bos']] = 1
        for i in range(config.max_length):
            output_tokens, h, c = self.inf_decoder_model.predict([target_seq] + states_value)
            states_value = [h, c]
            output_tokens = output_tokens.reshape(self.num_decoder_tokens)
            y_hat = np.argmax(output_tokens)
            if y_hat == 0:
                continue
            if inv_map[y_hat] is None:
                break
            else:
                sentence = sentence + inv_map[y_hat] + ' '
                target_seq = np.zeros((1, 1, 1500))
                target_seq[0, 0, y_hat] = 1
        return ' '.join(sentence.split()[:-1])

In [77]:
c_base =Video2Text()
c_base.load_inference_models()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_inputs (InputLayer)  [(None, 80, 4096)]       0         
                                                                 
 endcoder_lstm (LSTM)        [(None, 80, 512),         9439232   
                              (None, 512),                       
                              (None, 512)]                       
                                                                 
Total params: 9,439,232
Trainable params: 9,439,232
Non-trainable params: 0
_________________________________________________________________
Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_18 (InputLayer)          [(None, None, 1500)  0           []                              